# Simple RL code to teach the model to stand

## Setup jax enviroment and Load Model 

In [1]:
import numpy as np
import mediapy as media
import matplotlib.pyplot as plt


import os
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".60"

# Optionally, force JAX to preallocate memory.
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "true"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

# Setup environment variable for Nvidia GPU acceleration
os.environ['XLA_FLAGS'] = (
    # '--xla_gpu_enable_triton_softmax_fusion=true '
    '--xla_gpu_triton_gemm_any=True '
    # '--xla_gpu_enable_async_collectives=true '
    # '--xla_gpu_enable_latency_hiding_scheduler=true '
    '--xla_gpu_enable_highest_priority_async_stream=true '
    # '--xla_cpu_multi_thread_eigen=true intra_op_parallelism_threads=32'
)

backend = 'gpu'
# backend = 'METAL'
# backend = 'cpu'

import jax
jax.config.update('jax_platform_name', 'gpu')
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_lazy_compilation=false"
# Enable compliation catch
os.environ["JAX_COMPILATION_CACHE_DIR"] = "./jax_cache"
jax.config.update("jax_compilation_cache_dir", "./jax_cache")
jax.config.update("jax_persistent_cache_min_entry_size_bytes", 0)
jax.config.update("jax_persistent_cache_min_compile_time_secs", 2)
# jax.config.update("jax_explain_cache_misses", True)

from jax.experimental.compilation_cache import compilation_cache as cc
cc.set_cache_dir("./jax_cache")
# Debug Nan
# jax.config.update("jax_debug_nans", True)

from jax import numpy as jp
# More legible printing from numpy.
jp.set_printoptions(precision=4, suppress=True, linewidth=100)

import mujoco
import mujoco.mjx as mjx
from mujoco.mjx._src import scan
from mujoco.mjx._src import types

# More legible printing from numpy.
np.set_printoptions(precision=4, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

device = jax.devices(backend=backend)[0]

model_path = '/home/bugman/Currentwork/biomujoco_converter/converted/mjc/Gait2354/gait2354_cvt1.xml'

# Single step
mjx_step = jax.jit(mjx.step, backend=backend)



# mjx_multiple_steps = jax.jit(multiple_steps, backend=backend, )

In [2]:
from mujoco.mjx._src.biomtu import acceleration_mtu

mj_model = mujoco.MjModel.from_xml_path(model_path)
mjx_model = mjx.put_model(mj_model,device=device)

# Disable tendon
opt = mjx_model.opt.replace(disableflags = mjx_model.opt.disableflags |mujoco.mjtDisableBit.mjDSBL_PASSIVE)
mjx_model = mjx_model.replace(opt=opt)

mjx_data = mjx.make_data(mjx_model)
mj_data = mujoco.MjData(mj_model)

# Load the Keyframe
mjx_data = mjx_data.replace(qpos = mj_model.key_qpos[0])
mj_data.qpos = mj_model.key_qpos[0]

# Calculate equilibrum
mjx_data = acceleration_mtu.calc_equilibrium(mjx_model, mjx_data)
for i in range(10):
    mjx_data = mjx_step(mjx_model, mjx_data)

def print_all():
    print(mjx_model.biomtu_adr)
    print(mjx_model.mtu_wrap_objid)
    print(mjx_model.mtu_wrap_type)
    print(mjx_model.biomtu_fiso)
    print(mjx_model.biomtu_vmax)
    print(mjx_model.biomtu_ofl)
    print(mjx_model.biomtu_opa)
    print(mjx_model.biomtu_mass)
    print("-------Data--------")
    print("qpos:", mjx_data.qpos)
    print("mtu l:", mjx_data.biomtu.l)
    print("tendon l:", mjx_data.biomtu.tendon_l)
    print("fiber l :", mjx_data.biomtu.fiber_l)
    print("Muscle Bce:", mjx_data.biomtu.B_ce)
    print("Muscle vm:", mjx_data.biomtu.m)
    print("Fiber acc:", mjx_data.biomtu.fiber_acc)
    print("Fiber v:", mjx_data.biomtu.fiber_v)
    print("Biomtu h:", mjx_data.biomtu.h)
    print(mjx_data.biomtu.v)
    print(mjx_data.biomtu.h)  # The constant high of the muscle.
    print(mjx_data.biomtu.pennation_angle)
    print(mjx_data.biomtu.origin_body_id)
    print(mjx_data.biomtu.insertion_body_id)
    print("mtu act:", mjx_data.biomtu.act)
    # print(mjx_data.biomtu.j)
    print(mjx_data.qfrc_biomtu)
    print(mj_model.key_time)
    print(mj_model.key_qpos)
    print(mj_model.key_qvel)

print_all()

[  0   2   4   6   9  12  17  19  23  25  29  33  37  41  46  51  53  55  58  61  65  68  70  74
  77  79  81  83  86  89  94  96 100 102 106 110 114 118 123 128 130 132 135 138 142 145 147 151
 154 156 158 160 162 164]
[  0  58   1  59   2  60   3  86  87  61  88  89   4  62  90  91  92   5  63   6  64  65  93   7
  66   8  94  95  96   9  10  67  68  11  12  69  70  13  14  71  72  15  16  17  73  74  18  19
  20  75  76  21  77  22  78  23  24  79  25  80  97  81  82  83  98  84  85 104  99 105 100 101
 106 107 102 103 108  26 109  27 110  28 111  29 137 138 112 139 140  30 113 141 142 143  31 114
  32 115 116 144  33 117  34 145 146 147  35  36 118 119  37  38 120 121  39  40 122 123  41  42
  43 124 125  44  45  46 126 127  47 128  48 129  49  50 130  51 131 148 132 133 134 149 135 136
 155 150 156 151 152 157 158 153 154 159  52 161  53 162  54 163  55 164  56 165  57 166]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 

In [3]:
print(mjx_model.nbiomtu)
print(mjx_model.nq)
print(mjx_data.qpos)
print(mjx_data.qvel)

54
27
[ 0.0001  0.948  -0.     -0.002  -0.0002  0.0001 -0.0013 -0.0025 -0.0321 -0.0037 -0.3956  0.0127
 -0.1089  0.0696  0.2862 -0.0013 -0.0029 -0.0315 -0.0037 -0.3956  0.0126 -0.1087  0.0693  0.2855
  0.0027  0.0002 -0.0001]
[-0.0081 -0.2281 -0.0006 -0.5096 -0.0105  0.0095  0.5952 -0.2065 -2.3289  0.0002  0.0001 -0.0692
  1.3683  7.2302 10.6054  0.6013 -0.2312 -2.2837  0.0004  0.0001 -0.0896  1.4506  7.237  10.4265
  0.586   0.0117 -0.0093]


## Neural Network

In [4]:
import jax
import jax.numpy as jp
import flax
import flax.linen as nn
import optax

class Controller_NN(nn.Module):
    # It will output the mean and variance for each muscle's activation
    
    def setup(self):
        # Features means the output dimension# Single step
        self.nbiomtu = 54
        self.linear1 = nn.Dense(features=400)
        self.linear2 = nn.Dense(features=400)
        self.linear3 = nn.Dense(features=400)
        self.linear4 = nn.Dense(features=400)
        # The last layer will output the mean and logstd
        self.linear5 = nn.Dense(features=self.nbiomtu*2)
        
    
    def __call__(self, x, key):
        x = self.linear1(x)
        x = nn.relu(x)
        x = self.linear2(x)
        x = nn.relu(x)
        x = self.linear3(x)
        x = nn.relu(x)
        x = self.linear4(x)
        x = nn.relu(x)
        x = self.linear5(x)
        # The last layer of the neural requires samping
        mean = x[0:self.nbiomtu]
        logstd = x[self.nbiomtu:self.nbiomtu*2]
        std = jp.exp(logstd)
        samples = jp.clip(jax.random.normal(key)*std*0.1 + mean, 0 ,1)
        
        return samples, mean, logstd


# Test the neural network
control_model = Controller_NN()

print(control_model)
# Init the model
key = jax.random.key(66)
sub_keys = jax.random.split(key,1)
# The second parameter is the dommy input
params = control_model.init(key,jp.empty([1,mjx_model.nq*2]),sub_keys[0])
# print(params)
print(control_model.apply( params, jp.ones(mjx_model.nq*2), sub_keys[0]))
jit_nn_apply = jax.jit(lambda params,states,key : control_model.apply(params,states,key))

Controller_NN()
(Array([0.485 , 0.6272, 0.1041, 0.    , 0.5002, 0.    , 0.0713, 0.    , 0.    , 0.1078, 0.2488,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.1104, 0.1122, 0.    , 0.    ,
       0.    , 0.    , 0.0592, 0.    , 0.    , 0.    , 0.6122, 0.    , 0.3396, 0.    , 0.    ,
       0.    , 0.0524, 0.3564, 0.    , 0.0212, 0.    , 0.0724, 0.3618, 0.    , 0.    , 0.0428,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],      dtype=float32), Array([ 0.5507,  0.7117,  0.1663, -0.386 ,  0.572 ,  0.0428,  0.151 , -0.341 , -0.1592,  0.1468,
        0.3125, -0.506 , -0.098 , -0.1812, -0.2472, -0.1027, -0.2657, -0.0098,  0.1604,  0.1802,
       -0.4265, -0.2269, -0.2294, -0.0505,  0.1361,  0.0374,  0.0641, -0.0512,  0.6791, -0.3706,
        0.4092, -0.1719, -0.2406, -0.022 ,  0.1271,  0.4203,  0.0284,  0.0943, -0.4435,  0.1295,
        0.4245,  0.0444, -0.1952,  0.121 , -0.08  , -0.3598, -0.0973, -0.0134,  0.0238, -0.2089,
       -0

## Combine Neural Net and Simulation into one Jax function

In [5]:
# Multiple steps
def step_fn(carry, _):
    data, model = carry
    new_data = mjx.step(model, data)
    new_carry = (new_data, model)
    return new_carry, _

def multiple_steps(model, data):
    init_carry = (data, model)
    y, _ = jax.lax.scan(step_fn, init_carry, None, length=10)
    new_data = y[0]
    return new_data

def nn_mjx_one_step(nn_params, model, data, key):
    states = jp.concatenate([data.qpos, data.qvel])
    act = jit_nn_apply(nn_params, states, key)[0]
    # Generate the next key
    new_key = jax.random.split(key,1)[0]
    # mtu = data.biomtu
    # mtu = mtu.replace(act = act)
    # data = data.replace(biomtu = mtu)
    new_data = mjx.step(model, data)
    return new_data, new_key

@jax.jit
def nn_mjx_multi_steps(nn_params, model, data, key):
    def nn_step_fn(carry, _):
        data, key = carry
        new_data, new_key = nn_mjx_one_step(nn_params, model, data, key)
        new_carry = (new_data, new_key)
        # head_hight = new_data.xpos[2,2]
        # jax.debug.print("Head Height {0}",head_hight)
        return new_carry, _
    init_carry = (data, key)
    y, _ = jax.lax.scan(nn_step_fn, init_carry, None, length=10)
    new_data = y[0]
    new_key = y[1]
    return new_data, new_key

## Testing control model with neural networks

In [11]:
import mujoco.viewer
import time

previous_frame_time = time.time()
i = 0
key = jax.random.key(334)
with mujoco.viewer.launch_passive(mj_model, mj_data) as viewer:
    while viewer.is_running():
        # Update mjx_data from mj_data. The mj_data was modified by the viewer
        # mjx_data = mjx_data.replace(ctrl=mj_data.ctrl, xfrc_applied=mj_data.xfrc_applied)
        # Use the nerual network to generate ctrl signal
        # Generate key
        
        mjx_data = mjx_data.replace(xfrc_applied=jp.array(mj_data.xfrc_applied, dtype=jp.float32))
        mjx_data = mjx_data.replace(
            qpos= jp.array(mj_data.qpos, dtype=jp.float32),
            qvel= jp.array(mj_data.qvel, dtype=jp.float32),
            time = jp.array(mj_data.time, dtype=jp.float32))
        
        # Update mjx_model from mj_model
        mjx_model = mjx_model.tree_replace({
            'opt.gravity': jp.array(mj_model.opt.gravity, dtype=jp.float32),
            'opt.tolerance': jp.array(mj_model.opt.tolerance, dtype=jp.float32),
            'opt.ls_tolerance': jp.array(mj_model.opt.ls_tolerance, dtype=jp.float32),
            'opt.timestep': jp.array(mj_model.opt.timestep, dtype=jp.float32),
        })
        
        # mjx_data = mjx_step(mjx_model, mjx_data)
        mjx_data, key = nn_mjx_multi_steps(params, mjx_model, mjx_data, key)
        mjx.get_data_into(mj_data, mj_model, mjx_data)
        
        # Record the current time at the start of this frame
        current_frame_time = time.time()
    
        # Calculate the difference in time from the last frame
        time_between_frames = current_frame_time - previous_frame_time
    
        # Print the time between frames
        print(f"Time between frames: {time_between_frames} seconds")
        previous_frame_time = current_frame_time
        
        # print("ACT:", mjx_data.biomtu.act)
        # print(mjx_data.qpos)
        print(mjx_data.sensordata)
        # print(len(mjx_data.qvel))
        viewer.sync()

Time between frames: 6.814907550811768 seconds
[-0.1008 -0.0003  1.6299]


## Batched Random Init Model

In [6]:
def random_init(data, model, rng: jax.Array):
    nbiomtu = model.nq
    init_qpos = data.qpos
    init_qvel = data.qvel
    new_rng, rng1, rng2 = jax.random.split(rng, 3)
    # Qpos_1 is the vertical position
    random_qpos = init_qpos + jax.random.uniform(rng1, [nbiomtu], minval=jp.array(-1.0, dtype=jp.float32), maxval=jp.array(1.0, dtype=jp.float32))*0.001
    random_qvel = init_qvel + jax.random.uniform(rng2, [nbiomtu], minval=jp.array(-1.0, dtype=jp.float32), maxval=jp.array(1.0, dtype=jp.float32))*0.001
    newdata = data.replace(qpos=random_qpos)
    newdata = newdata.replace(qvel=random_qvel)
    newdata = mjx.forward(mjx_model, newdata)
    # print('data:',data.qpos, data.qvel)
    # Calculate equilibrum
    # newdata = acceleration_mtu.calc_equilibrium(mjx_model, newdata)
    # newdata = mjx_step(mjx_model, newdata)
    return newdata, new_rng



## Test Init

In [7]:
batch_size = 50
seed = 2024
key = jax.random.key(seed)
rngs = jax.random.split(key, batch_size) 
vrandom_init_lower = jax.jit(jax.vmap(random_init, in_axes=(None, None, 0), out_axes=0)).lower(mjx_data, mjx_model, rngs)
vrandom_init = vrandom_init_lower.compile()


In [8]:
mjx_data_batch, rngs = vrandom_init(mjx_data, mjx_model, rngs)
print(mjx_data_batch.biomtu)

Biomtu(fiber_l=Array([[0.0443, 0.0782, 0.0631, ..., 0.0991, 0.1335, 0.1335],
       [0.0443, 0.0782, 0.0631, ..., 0.0991, 0.1335, 0.1335],
       [0.0443, 0.0782, 0.0631, ..., 0.0991, 0.1335, 0.1335],
       ...,
       [0.0443, 0.0782, 0.0631, ..., 0.0991, 0.1335, 0.1335],
       [0.0443, 0.0782, 0.0631, ..., 0.0991, 0.1335, 0.1335],
       [0.0443, 0.0782, 0.0631, ..., 0.0991, 0.1335, 0.1335]], dtype=float32), fiber_v=Array([[0.0292, 0.0001, 0.1456, ..., 0.0053, 0.018 , 0.0149],
       [0.0268, 0.0001, 0.1456, ..., 0.0031, 0.0114, 0.0136],
       [0.0274, 0.0001, 0.1456, ..., 0.0081, 0.0131, 0.0217],
       ...,
       [0.0271, 0.0001, 0.1456, ..., 0.0033, 0.0172, 0.0117],
       [0.0274, 0.0001, 0.1456, ..., 0.0029, 0.0163, 0.0118],
       [0.0272, 0.0001, 0.1456, ..., 0.0031, 0.0147, 0.0134]], dtype=float32), act=Array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       

## Reward Function
get reward at the same time calculate the mjx_data

In [9]:
# from functools import partial
def nn_mjx_one_step(nn_params, model, data, key):
    states = jp.concatenate([data.qpos, data.qvel])
    act = jit_nn_apply(nn_params, states, key)[0]
    # Generate the next key
    new_key = jax.random.split(key,1)[0]
    data = data.replace(biomtu = data.biomtu.replace(act=act))
    new_data = mjx.step(model, data)
    return new_data, new_key

def nn_step_fn(carry, _):
    data, key, nn_params, model = carry
    new_data, new_key = nn_mjx_one_step(nn_params, model, data, key)
    new_carry = (new_data, new_key, nn_params, model)
    head_hight = new_data.sensordata[2]
    # jax.debug.print("Head Height {0}",head_hight)
    # return new_carry, (head_hight-1.63)**2
    return new_carry, -head_hight

def decay_sum_scan(x, decay):
    def f(sxtm1, xt):
        b = xt + decay * sxtm1
        return b, b
    return jax.lax.scan(f, jp.zeros(x.shape[1:]), x)[1]

# This function return the loss as well as the new data
def reward_n_step(nn_params, model, data, key):
    repeat_n = 150  # Simulate for 0.1s
    carry, head_heights = jax.lax.scan(nn_step_fn, (data, key, nn_params, model), None, repeat_n)
    # loss = decay_sum_scan(jp.flip(head_heights), 0.95)[repeat_n -1]
    loss = decay_sum_scan(head_heights, 0.99)[repeat_n -1]
    new_data = carry[0]
    return loss, new_data

def batch_reward(nn_params, batched_data, keys, model):
    batch_loss, batch_new_data = jax.vmap(reward_n_step, (None, None, 0, 0))(nn_params, model, batched_data, keys)
    return jp.mean(batch_loss), batch_new_data 


batch_size = 50
seed = 2024
key = jax.random.key(seed)
rngs = jax.random.split(key, batch_size) 
mjx_data_batch, rngs = vrandom_init(mjx_data, mjx_model, rngs)

print("start lower")

jit_batch_reward_lower = jax.jit(batch_reward).lower(params, mjx_data_batch, rngs, mjx_model)
jit_batch_reward_grad_lower = jax.jit(jax.grad(batch_reward,has_aux=True)).lower(params, mjx_data_batch, rngs, mjx_model)


start lower


In [10]:
print("start compiling ")

jit_batch_reward = jit_batch_reward_lower.compile()
jit_batch_reward_grad = jit_batch_reward_grad_lower.compile()

start compiling 


In [11]:
# Generate batched data
batch_size = 50
seed = 2024
key = jax.random.key(seed)
rngs = jax.random.split(key, batch_size) 
mjx_data_batch, rngs = vrandom_init(mjx_data, mjx_model, rngs)

r,d = jit_batch_reward(params, mjx_data_batch, rngs, mjx_model)
print(r)
print("Calculating reward grad")
# pr = jax.make_jaxpr(jit_batch_reward_grad)(params, mjx_data_batch, rngs, mjx_model)
g,d = jit_batch_reward_grad(params, mjx_data_batch, rngs, mjx_model)
print(g)

-114.62678
Calculating reward grad
{'params': {'linear1': {'bias': Array([ 0.0028, -0.0144,  0.3017,  0.0614,  0.7004,  0.0057, -0.1587, -0.1972,  0.122 ,  0.1417,
        0.1881, -0.3288,  0.3094, -0.0117, -0.3451, -0.0005, -0.0346, -0.2676, -0.0248,  0.0286,
       -0.0848,  0.2032, -0.0095, -0.2534,  0.0138,  0.2306, -0.0443,  0.0498, -0.465 , -0.5352,
       -0.0517, -0.2632,  0.0424,  0.1873, -0.1013,  0.1315, -0.1771, -0.2219,  0.0429, -0.559 ,
       -0.0524,  0.046 ,  0.0907,  0.0531, -0.292 ,  0.0016,  0.0005,  0.5009,  0.01  , -0.0196,
        0.4683,  0.3742,  0.0647, -0.3175,  0.0014, -0.0484, -0.0264,  0.0495, -0.0618, -0.2124,
        0.0067, -0.1469, -0.4852, -0.1307,  0.0644,  0.0093, -0.3483,  0.0662,  0.0002, -0.1589,
       -0.0695,  0.1086,  0.0274, -0.1804,  0.1128,  0.1903, -0.0123, -0.0002, -0.0965,  0.0008,
        0.1815, -0.2844,  0.1709, -0.1292, -0.0296,  0.9195, -0.4495, -0.0715, -0.002 , -0.3585,
        0.0229,  0.0654,  0.061 ,  0.0438, -0.2406,  0.1133,

## Reset part of the data based on the condition

## Train the NN

In [19]:
seed = 2024
mjx_data = mjx.make_data(mjx_model)
mj_data = mujoco.MjData(mj_model)

# Load the Keyframe
mjx_data = mjx_data.replace(qpos = mj_model.key_qpos[0])
mj_data.qpos = mj_model.key_qpos[0]
# Calculate equilibrum
mjx_data = acceleration_mtu.calc_equilibrium(mjx_model, mjx_data)
mjx_data = mjx_step(mjx_model, mjx_data)
mjx_data_batch, rngs = vrandom_init(mjx_data, mjx_model, rngs)

In [20]:
params = control_model.init(key,jp.empty([1,mjx_model.nq*2]),sub_keys[0])
tx = optax.adam(learning_rate=5e-4)
opt_state = tx.init(params)


In [21]:

for i in range(400):
    print(i)
    # generate random mjx_data
    mjx_data_batch, rngs = vrandom_init(mjx_data, mjx_model, rngs)
    # print(rngs[0])
    r,d = jit_batch_reward(params, mjx_data_batch, rngs, mjx_model)
    print(r)
    g,d = jit_batch_reward_grad(params, mjx_data_batch, rngs, mjx_model)
    updates, opt_stats = tx.update(g, opt_state)
    params = optax.apply_updates(params, updates)
    print("params updated")

0
-121.30117
params updated
1
-122.77068
params updated
2
-123.39816
params updated
3
-121.787476
params updated
4
-126.47096
params updated
5
-124.439316
params updated
6
-128.38518
params updated
7
-127.09404
params updated
8
-129.13385
params updated
9
-129.55475
params updated
10
-129.31145
params updated
11
-129.35469
params updated
12
-129.2753
params updated
13
-129.95633
params updated
14
-129.99629
params updated
15
-129.86938
params updated
16
-129.69736
params updated
17
-129.95035
params updated
18
-130.20436
params updated
19
-129.5398
params updated
20
-130.09798
params updated
21
-130.18988
params updated
22
-130.16193
params updated
23
-130.15198
params updated
24
-130.05634
params updated
25
-130.31749
params updated
26
-129.99695
params updated
27
-130.31923
params updated
28
-130.26031
params updated
29
-130.3762
params updated
30
-129.60425
params updated
31
-130.23914
params updated
32
-130.43468
params updated
33
-130.44873
params updated
34
-130.57385
params upda

KeyboardInterrupt: 

## Test Train

In [22]:
jit_nn_mjx_one_step = jax.jit(nn_mjx_one_step)


In [26]:
import mujoco.viewer
import time


mjx_data = mjx.make_data(mjx_model)
mj_data = mujoco.MjData(mj_model)

# Load the Keyframe
mjx_data = mjx_data.replace(qpos = mj_model.key_qpos[0])
mj_data.qpos = mj_model.key_qpos[0]

# Calculate equilibrum
mjx_data = acceleration_mtu.calc_equilibrium(mjx_model, mjx_data)
mjx_data = mjx_step(mjx_model, mjx_data)

previous_frame_time = time.time()
i = 0
key = jax.random.key(seed)
with mujoco.viewer.launch_passive(mj_model, mj_data) as viewer:
    while viewer.is_running():
        i += 1
        if(i ==4):
            time.sleep(2)
            # pass
        # Update mjx_data from mj_data. The mj_data was modified by the viewer
        # mjx_data = mjx_data.replace(ctrl=mj_data.ctrl, xfrc_applied=mj_data.xfrc_applied)
        # Use the nerual network to generate ctrl signal
        # Generate key
        
        mjx_data = mjx_data.replace(xfrc_applied=jp.array(mj_data.xfrc_applied, dtype=jp.float32))
        
        mjx_data = mjx_data.replace(
            qpos= jp.array(mj_data.qpos, dtype=jp.float32),
            qvel= jp.array(mj_data.qvel, dtype=jp.float32),
            time = jp.array(mj_data.time, dtype=jp.float32))
        
        # Update mjx_model from mj_model
        mjx_model = mjx_model.tree_replace({
            'opt.gravity': jp.array(mj_model.opt.gravity, dtype=jp.float32),
            'opt.tolerance': jp.array(mj_model.opt.tolerance, dtype=jp.float32),
            'opt.ls_tolerance': jp.array(mj_model.opt.ls_tolerance, dtype=jp.float32),
            'opt.timestep': jp.array(mj_model.opt.timestep, dtype=jp.float32),
        })
        
        # mjx_data = mjx_step(mjx_model, mjx_data)
        mjx_data, key = jit_nn_mjx_one_step(params, mjx_model, mjx_data, key)
        # mjx_data, key = nn_mjx_multi_steps(params, mjx_model, mjx_data, key)
        
        mjx.get_data_into(mj_data, mj_model, mjx_data)
        
        # Record the current time at the start of this frame
        current_frame_time = time.time()
    
        # Calculate the difference in time from the last frame
        time_between_frames = current_frame_time - previous_frame_time
    
        # Print the time between frames
        print(f"Time between frames: {time_between_frames} seconds")
        previous_frame_time = current_frame_time
        
        # print("ACT:", mjx_data.biomtu.act)
        # print(mjx_data.qpos)
        print(mjx_data.sensordata[2])
        # print(len(mjx_data.qvel))
        viewer.sync()

Time between frames: 0.2136073112487793 seconds
1.6315
Time between frames: 0.008671998977661133 seconds
1.6314898
Time between frames: 0.00850367546081543 seconds
1.6314723
Time between frames: 2.009376287460327 seconds
1.6314639
Time between frames: 0.008504867553710938 seconds
1.6314588
Time between frames: 0.007045269012451172 seconds
1.6314232
Time between frames: 0.0067729949951171875 seconds
1.631313
Time between frames: 0.0069124698638916016 seconds
1.6311321
Time between frames: 0.0070383548736572266 seconds
1.6308845
Time between frames: 0.012320756912231445 seconds
1.6305733
Time between frames: 0.006772518157958984 seconds
1.630218
Time between frames: 0.006949186325073242 seconds
1.6298326
Time between frames: 0.00711369514465332 seconds
1.6294544
Time between frames: 0.0064313411712646484 seconds
1.6290896
Time between frames: 0.006700754165649414 seconds
1.6287401
Time between frames: 0.006792306900024414 seconds
1.6284279
Time between frames: 0.006945371627807617 second

## Test Gradient for activation

In [ ]:
n = mjx_model.nbiomtu
act = jp.ones(n)*0.1

In [ ]:
# jax.config.update("jax_debug_nans", True)
for i in range(10):
    g = jit_goal_grad(act, mjx_data, mjx_model)
    # g = goal_grad(act, mjx_data, mjx_model)
print(g)

In [ ]:
v_act = jp.ones((50,n))*0.3
for i in range(5):
    v_g = jit_v_goal_grad(v_act,mjx_data,mjx_model)
print(v_g)

In [ ]:
for i in range(10):
    reward = jit_goal(act, mjx_data, mjx_model)
print(reward)

In [ ]:
act = jp.ones(n)*0.0
mtu = mjx_data.biomtu
mtu = mtu.replace(act = act)
test_d = mjx_data
test_d = test_d.replace(biomtu = mtu)
for i in range(100):
    test_d = mjx_step(mjx_model, test_d)
print(test_d.xipos)

In [ ]:
print(test_d.biomtu.f_se)
print(test_d.biomtu.f_ce)
print(test_d.biomtu.l)
print(test_d.biomtu.tendon_l - mjx_model.biomtu_tendon_slack_l)
print(test_d.biomtu.fiber_l - mjx_model.biomtu_ofl)
print(test_d.biomtu.fiber_l)
print(test_d.biomtu.act)

In [ ]:
print(mjx_data.xpos)

In [ ]:
mjx_model.nbiomtu

In [ ]:
batch_size = 40
seed = 2024
key = jax.random.key(seed)
rngs = jax.random.split(key, batch_size) 
mjx_data_batch, rngs2 = vrandom_init(mjx_data, mjx_model, rngs)

In [ ]:
print(rngs.shape)
print(rngs2.shape)
print(mjx_data_batch.biomtu.l.shape)

In [ ]:
from functools import *

# A normal function
def test_f(input):
    a,b,c = input
    return 100 * a + 10 * b + c, a*10

test_g = jax.jit(jax.grad(test_f, has_aux=True))


r = test_g((1.,3.,4.))

print(r)

In [ ]:
mjx_data.xfrc_applied.shape

In [ ]:
a = jp.ones(54)
b = jp.zeros(30)
c = jp.pad(a,15)
print(c)

In [2]:
import jax.numpy as jp
dones = jp.array([1,0,1,1,0,0])
a = jp.array([250]*6)
b = jp.array([233]*6)
c = jp.where(dones, a, b)
print(c)

[250 233 250 250 233 233]
